In [1]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

from IPython.display import display

from konlpy.tag import Kkma
from konlpy.tag import Twitter
from konlpy.utils import pprint

from nltk.tokenize import sent_tokenize, word_tokenize

import nltk
nltk.download('punkt')

import queue
import threading
import subprocess
import multiprocessing

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /Users/sw_lee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Init files

train_file_name = 'train'
test_file_name = 'test'

full_dir_format = 'dataset_1/{}.txt'
processed_file_dir_format = 'dataset_1/processed_{}.txt'

train_file_dir = full_dir_format.format(train_file_name)
test_file_dir = full_dir_format.format(test_file_name)
processed_train_file_dir = processed_file_dir_format.format(train_file_name)
processed_test_file_dir = processed_file_dir_format.format(test_file_name)
processed_test_file_dir

'dataset_1/processed_test.txt'

In [3]:
# Open files

train_file = open(train_file_dir, 'r', encoding='utf8')
processed_train_file = open(processed_train_file_dir, 'r+', encoding='utf8')

test_file = open(test_file_dir, 'r', encoding='utf8')
processed_test_file = open(processed_test_file_dir, 'r+', encoding='utf8')

In [4]:
# Preprocess files

for line in train_file:
    processed_train_file.write(line.replace('\t', '\s', line.count('\t') - 1))

train = pd.read_csv(processed_train_file_dir, delimiter='\t', header=None, names=['comment_text', 'type'])
train = train.sample(frac=1).reset_index(drop=True)
    
for line in test_file:
    processed_test_file.write(line.replace('\t', '\s', line.count('\t') - 1))

test = pd.read_csv(processed_test_file_dir, delimiter='\t', header=None, names=['comment_text', 'type'])
test = test.sample(frac=1).reset_index(drop=True)

In [5]:
# concat = pd.concat([train, test], axis=0, keys=['train','test'])
# concat_test = pd.concat([train[:100], test[:50]], axis=0, keys=['train','test'])

# # Change categorical to index codes
# concat['type'] = pd.Categorical(concat['type'])
# concat_test['type'] = pd.Categorical(concat_test['type'])

# # Y is in [0, 1] where 1 is spam and 0 is normal
# y_concat = concat['type'].astype('category').cat.codes
# y_concat_test = concat_test['type'].astype('category').cat.codes

# Change categorical to index codes
# train = train[:10000]
# test = test[:5000]

train['type'] = pd.Categorical(train['type'])
test['type'] = pd.Categorical(test['type'])

# Y is in [0, 1] where 1 is spam and 0 is normal
y_t = train['type'].astype('category').cat.codes
y_te = test['type'].astype('category').cat.codes

# Get list of comments
list_sentences_train = train["comment_text"]
list_sentences_test = test["comment_text"]

In [6]:
# list_concat = concat['comment_text']
# list_concat_test = concat_test['comment_text']

In [7]:
import time
twitter = Twitter()
def thread_pos_tagging(data_list, tag):
    q = queue.Queue()
    for i, v in enumerate(data_list):
        q.put((i, v))
        
    result = [None] * len(data_list)
    def _tagging(result):
        while True:
            i, v = q.get()
            result[i] = " ".join(["".join(w) for w, t in tag.pos(v)])
            q.task_done()

    cpus=multiprocessing.cpu_count() #detect number of cores
    print("Creating %d threads" % cpus)
    for i in range(cpus):
        t = threading.Thread(target=_tagging, args=(result,))
        t.daemon = True
        t.start()

    q.join()
    return result

start = time.time()
doc0_t = thread_pos_tagging(list_sentences_train, twitter)
print('execution time: %.2f' % (time.time() - start))

# doc0_t = [" ".join(["".join(w) for w, t in twitter.pos(s)]) for s in list_sentences_train]
# doc0_te = [" ".join(["".join(w) for w, t in twitter.pos(s)]) for s in list_sentences_test]

/Users/sw_lee/.pyenv/versions/3.6.2/envs/cs409-3.6.2/lib/python3.6/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


Creating 12 threads
execution time: 360.18


In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(doc0_t)
list_tokenized_train = tokenizer.texts_to_sequences(doc0_t)
list_tokenized_test = tokenizer.texts_to_sequences(doc0_te)

NameError: name 'doc0_te' is not defined

In [ ]:
print(len(list_tokenized_train))
print(len(list_tokenized_test))

In [237]:
maxlen = max([len(x) - 1 for x in doc])
vocab_size = len(tokenizer.word_index) + 1
maxlen, vocab_size

(75, 26874)

In [238]:
# # Tokenize the comments

# max_features = 20000
# tokenizer = Tokenizer(num_words=max_features, char_level=True)

# tokenizer.fit_on_texts(list(list_sentences_train))
# list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
# list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

In [239]:
# Pad sequences

X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
X_te[:1]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)

In [240]:
# Input layer

inp = Input(shape=(maxlen, ))

In [241]:
# Embed layer

embed_size = 128
x = Embedding(vocab_size, embed_size)(inp)

In [242]:
# LSTM layer

x = LSTM(200, return_sequences=False, name='lstm_layer')(x)

In [243]:
# # Max Pooling layer
 
# x = GlobalMaxPool1D()(x)

In [244]:
# Dropout layer (1/2)

x = Dropout(0.1)(x)

In [245]:
# Dense Layer

x = Dense(50, activation="relu")(x)

In [246]:
# Droupout layer (2/2)

x = Dropout(0.1)(x)

In [247]:
# Final Dense layer

x = Dense(1, activation="sigmoid")(x)

In [248]:
# Optimizer

model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [249]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 75)                0         
_________________________________________________________________
embedding_12 (Embedding)     (None, 75, 128)           3439872   
_________________________________________________________________
lstm_layer (LSTM)            (None, 200)               263200    
_________________________________________________________________
dropout_25 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 50)                10050     
_________________________________________________________________
dropout_26 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 51        
Total para

In [250]:
# Train model

batch_size = 100
epochs = 5
model.fit(X_t, y_t, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Train on 9000 samples, validate on 1000 samples
Epoch 1/5
9000/9000 [==============================] - 17s 2ms/step - loss: 0.3046 - acc: 0.8953 - val_loss: 0.0636 - val_acc: 0.9800
Epoch 2/5
9000/9000 [==============================] - 16s 2ms/step - loss: 0.0240 - acc: 0.9938 - val_loss: 0.0587 - val_acc: 0.9830
Epoch 3/5
9000/9000 [==============================] - 15s 2ms/step - loss: 0.0060 - acc: 0.9986 - val_loss: 0.0595 - val_acc: 0.9800
Epoch 4/5
9000/9000 [==============================] - 15s 2ms/step - loss: 0.0013 - acc: 0.9996 - val_loss: 0.0705 - val_acc: 0.9810
Epoch 5/5
9000/9000 [==============================] - 16s 2ms/step - loss: 4.7848e-04 - acc: 1.0000 - val_loss: 0.0847 - val_acc: 0.9770


In [251]:

scores = model.evaluate(X_te, y_te, verbose=1)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

5000/5000 [==============================] - 3s 519us/step
acc: 68.36%


In [229]:
test_text = "못 보신 분들 \"마이티빙\"에서 무료로 보세요. 가입필요 없음."

tokenized = tokenizer.texts_to_sequences([test_text])

test_sequence = pad_sequences(tokenized, maxlen=maxlen)

model.predict(test_sequence)

array([[0.19465198]], dtype=float32)

In [274]:
def a():
    b = [None] * 10
    index = 0
    threads = []
    def _b(index):
        b[index] = index
        index += 1
    for i in range(10):
        threads.append(threading.Thread(target=_b,
                                        args=(i,)))
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
    return b
a()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [262]:
a = [None] * 10
a

[None, None, None, None, None, None, None, None, None, None]

In [1]:
a = [None] * 100000

In [14]:
import queue
import threading
import multiprocessing
import subprocess
import time

start = time.time()
a = [None] * 100000

q = queue.Queue()
for i in range(100000): #put 30 tasks in the queue
    q.put(i)

def worker(a):
    while True:
        item = q.get()
        #execute a task: call a shell program and wait until it completes
        a[item] = item
        q.task_done()

cpus=multiprocessing.cpu_count() #detect number of cores
print("Creating %d threads" % cpus)
for i in range(cpus):
     t = threading.Thread(target=worker, args=(a,))
     t.daemon = True
     t.start()

q.join()
print('execution time: %.2f' % (time.time() - start))

Creating 12 threads
execution time: 0.69


In [12]:
a[9999]

9999

In [15]:
start = time.time()

a = [None] * 100000
for i in range(100000):
    a[i] = i
print('execution time: %.2f' % (time.time() - start))    

execution time: 0.01
